# Задача

Реализовать класс `MyBinaryLogisticRegression` для работы с логистической регрессией. Обеспечить возможность использования `l1`, `l2` и `l1l2` регуляризации и реализовать слудующие методы решения оптимизационной задачи:

*   Градиентный спуск
*   Стохастический градиентный спуск
*   Метод Ньютона

Обосновать применимость/не применимость того или иного метода оптимизации в случае использованного типа регуляризации.



In [35]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

In [23]:
class MyBinaryLogisticRegression:
    def __init__(
        self,
        lr=0.01,
        n_iter=1000,
        optimizer="gd",
        penalty=None,
        lambda1=0.0,
        lambda2=0.0,
        batch_size=32,
        tol=1e-6,
        random_state=None
    ):
        self.lr = lr
        self.n_iter = n_iter
        self.optimizer = optimizer
        self.penalty = penalty
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        self.batch_size = batch_size
        self.tol = tol
        self.random_state = random_state

        self.coef_ = None
        self.intercept_ = None
        self.feature_names_in_ = None
        self.loss_history_ = []

        if random_state is not None:
            np.random.seed(random_state)

    def _sigmoid(self, z):
        z = np.clip(z, -500, 500)
        return 1 / (1 + np.exp(-z))

    def _add_intercept(self, X):
        return np.hstack([np.ones((X.shape[0], 1)), X])

    def _loss_grad(self, X, y, coef_full):
        n_samples = X.shape[0]

        z = X @ coef_full
        y_pred = self._sigmoid(z)

        grad = X.T @ (y_pred - y) / n_samples

        if self.penalty == "l1":
            grad[1:] += self.lambda1 * np.sign(coef_full[1:])
        elif self.penalty == "l2":
            grad[1:] += 2 * self.lambda2 * coef_full[1:]
        elif self.penalty == "l1l2":
            grad[1:] += (self.lambda1 * np.sign(coef_full[1:]) +
                        2 * self.lambda2 * coef_full[1:])

        return grad

    def _loss(self, X, y, coef_full):
        n_samples = X.shape[0]

        z = X @ coef_full
        y_pred = self._sigmoid(z)
        loss = -np.mean(y * np.log(y_pred + 1e-15) +
                       (1 - y) * np.log(1 - y_pred + 1e-15))

        if self.penalty == "l1":
            reg = self.lambda1 * np.sum(np.abs(coef_full[1:]))
        elif self.penalty == "l2":
            reg = self.lambda2 * np.sum(coef_full[1:]**2)
        elif self.penalty == "l1l2":
            reg = (self.lambda1 * np.sum(np.abs(coef_full[1:])) +
                  self.lambda2 * np.sum(coef_full[1:]**2))
        else:
            reg = 0

        return loss + reg

    def fit(self, X, y):
        if isinstance(X, pd.DataFrame):
            self.feature_names_in_ = X.columns.tolist()
            X_np = X.values
        else:
            X_np = X
            self.feature_names_in_ = None

        y_np = y.values.reshape(-1) if isinstance(y, pd.DataFrame) else y.reshape(-1)

        X_with_intercept = self._add_intercept(X_np)
        n_features = X_with_intercept.shape[1]

        coef_full = np.zeros(n_features)

        if self.optimizer == "gd":
            for epoch in range(self.n_iter):
                grad = self._loss_grad(X_with_intercept, y_np, coef_full)
                coef_full -= self.lr * grad

                loss = self._loss(X_with_intercept, y_np, coef_full)
                self.loss_history_.append(loss)

                if epoch > 0 and abs(self.loss_history_[-1] - self.loss_history_[-2]) < self.tol:
                    break

        elif self.optimizer == "sgd":
            n_samples = X_with_intercept.shape[0]

            for epoch in range(self.n_iter):
                indices = np.random.permutation(n_samples)
                X_shuffled = X_with_intercept[indices]
                y_shuffled = y_np[indices]

                epoch_loss = 0

                for i in range(0, n_samples, self.batch_size):
                    end_idx = min(i + self.batch_size, n_samples)
                    X_batch = X_shuffled[i:end_idx]
                    y_batch = y_shuffled[i:end_idx]

                    grad = self._loss_grad(X_batch, y_batch, coef_full)
                    coef_full -= self.lr * grad

                    epoch_loss += self._loss(X_batch, y_batch, coef_full)

                self.loss_history_.append(epoch_loss / (n_samples / self.batch_size))

                self.lr *= 0.999

                if epoch > 10 and abs(self.loss_history_[-1] - self.loss_history_[-2]) < self.tol:
                    break

        elif self.optimizer == "newton":
            if self.penalty == "l1":
                print("Предупреждение: метод Ньютона может работать некорректно с L1 регуляризацией")

            for epoch in range(self.n_iter):
                grad = self._loss_grad(X_with_intercept, y_np, coef_full)

                y_pred = self._sigmoid(X_with_intercept @ coef_full)
                W = np.diag(y_pred * (1 - y_pred))
                H = X_with_intercept.T @ W @ X_with_intercept / X_with_intercept.shape[0]

                if self.penalty in ("l2", "l1l2"):
                    reg_matrix = np.eye(n_features) * 2 * self.lambda2
                    reg_matrix[0, 0] = 0
                    H += reg_matrix

                try:
                    delta = np.linalg.solve(H, grad)
                except np.linalg.LinAlgError:
                    delta = np.linalg.pinv(H) @ grad

                coef_full -= delta

                loss = self._loss(X_with_intercept, y_np, coef_full)
                self.loss_history_.append(loss)

                if epoch > 0 and abs(self.loss_history_[-1] - self.loss_history_[-2]) < self.tol:
                    break

                if np.linalg.norm(delta) > 100:
                    print("Слишком большой шаг в методе Ньютона, уменьшаем")
                    break

        else:
            raise ValueError(f"Неизвестный оптимизатор: {self.optimizer}")

        self.intercept_ = coef_full[0]
        self.coef_ = coef_full[1:]

        return self

    def predict_proba(self, X):
        if isinstance(X, pd.DataFrame):
            X = X.values

        z = self.intercept_ + X @ self.coef_
        return self._sigmoid(z)

    def predict(self, X, threshold=0.5):
        proba = self.predict_proba(X)
        return (proba >= threshold).astype(int)

    def score(self, X, y):
        y_pred = self.predict(X)
        y_true = y.values.reshape(-1) if isinstance(y, pd.DataFrame) else y.reshape(-1)
        return np.mean(y_pred == y_true)

Продемонстрировать применение реализованного класса на датасете про пингвинов (целевая переменная — вид пингвина). Рассмотреть все возможные варианты (регуляризация/оптимизация). Для категориального признака `island` реализовать самостоятельно преобразование `Target Encoder`, сравнить результаты классификации с `one-hot`. В качестве метрики использовать `f1-score`.

In [34]:
df = pd.read_csv('penguins_binary_classification.csv')

In [2]:
df.shape

(274, 7)

In [3]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,2007
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,2007
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,2007
3,Adelie,Torgersen,36.7,19.3,193.0,3450.0,2007
4,Adelie,Torgersen,39.3,20.6,190.0,3650.0,2007


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 274 entries, 0 to 273
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   species            274 non-null    object 
 1   island             274 non-null    object 
 2   bill_length_mm     274 non-null    float64
 3   bill_depth_mm      274 non-null    float64
 4   flipper_length_mm  274 non-null    float64
 5   body_mass_g        274 non-null    float64
 6   year               274 non-null    int64  
dtypes: float64(4), int64(1), object(2)
memory usage: 15.1+ KB


In [5]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,year
count,274.000000,274.000000,274.000000,274.000000,274.000000
mean,42.702920,16.836131,202.178832,4318.065693,2008.043796
std,5.195566,2.013410,15.047938,835.933105,0.806281
min,32.100000,13.100000,172.000000,2850.000000,2007.000000
25%,38.350000,15.000000,190.000000,3600.000000,2007.000000
50%,42.000000,17.000000,198.000000,4262.500000,2008.000000
75%,46.675000,18.500000,215.000000,4950.000000,2009.000000
max,59.600000,21.500000,231.000000,6300.000000,2009.000000


In [6]:
df.isnull().sum()

,0
species,0
island,0
bill_length_mm,0
bill_depth_mm,0
flipper_length_mm,0
body_mass_g,0
year,0


In [8]:
for column in df.columns:
    unique_vals = df[column].nunique()
    print(f"   {column}: {unique_vals} уникальных значений")
    if unique_vals < 10:
        print(f"      {df[column].unique()}")

   species: 2 уникальных значений
      ['Adelie' 'Gentoo']
   island: 3 уникальных значений
      ['Torgersen' 'Biscoe' 'Dream']
   bill_length_mm: 146 уникальных значений
   bill_depth_mm: 78 уникальных значений
   flipper_length_mm: 54 уникальных значений
   body_mass_g: 89 уникальных значений
   year: 3 уникальных значений
      [2007 2008 2009]


In [9]:
df['species'].value_counts()

,count
species,
Adelie,151
Gentoo,123


In [12]:
df['target'] = (df['species'] == 'Gentoo').astype(int)

In [13]:
print(f"\nБаланс классов:")
print(f"   Класс 0: {(df['target'] == 0).sum()/len(df):.2%}")
print(f"   Класс 1: {(df['target'] == 1).sum()/len(df):.2%}")


Баланс классов:
   Класс 0: 55.11%
   Класс 1: 44.89%


In [15]:
class TargetEncoder:

    def __init__(self, smoothing=10):
        """
        Parameters:
        -----------
        smoothing : параметр сглаживания (чем больше, тем больше сглаживание)
        """
        self.smoothing = smoothing
        self.encoding_dict = {}
        self.global_mean = None

    def fit(self, X: pd.Series, y: pd.Series):
        """Обучение энкодера на данных"""
        self.global_mean = y.mean()

        print(f"Глобальное среднее целевой переменной: {self.global_mean:.4f}")

        for category in X.unique():
            mask = X == category
            category_mean = y[mask].mean()
            category_count = mask.sum()

            # Применяем сглаживание
            smoothed_mean = (category_mean * category_count + self.global_mean * self.smoothing) / \
                           (category_count + self.smoothing)

            self.encoding_dict[category] = smoothed_mean

            print(f"  Категория '{category}':")
            print(f"    Количество: {category_count}")
            print(f"    Среднее по категории: {category_mean:.4f}")
            print(f"    Сглаженное среднее: {smoothed_mean:.4f}")

        return self

    def transform(self, X: pd.Series) -> np.ndarray:
        """Преобразование категориального признака"""
        result = []
        for value in X:
            if value in self.encoding_dict:
                result.append(self.encoding_dict[value])
            else:
                # Для неизвестных категорий используем глобальное среднее
                result.append(self.global_mean)
        return np.array(result).reshape(-1, 1)

    def fit_transform(self, X: pd.Series, y: pd.Series) -> np.ndarray:
        """Обучение и преобразование за один шаг"""
        self.fit(X, y)
        return self.transform(X)

In [17]:
def prepare_data_with_target_encoder(df, test_size=0.2, random_state=42):

    numerical_features = ['bill_length_mm', 'bill_depth_mm',
                         'flipper_length_mm', 'body_mass_g']

    categorical_feature = 'island'

    y = df['target'].values

    target_encoder = TargetEncoder(smoothing=5)
    island_encoded = target_encoder.fit_transform(df[categorical_feature], y)

    X_numerical = df[numerical_features].values
    X = np.hstack([X_numerical, island_encoded])

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    feature_names = numerical_features + ['island_target_encoded']

    print(f"\nИтоговые признаки: {feature_names}")
    print(f"X_train: {X_train_scaled.shape}")
    print(f"X_test: {X_test_scaled.shape}")

    return {
        'X_train': X_train_scaled,
        'X_test': X_test_scaled,
        'y_train': y_train,
        'y_test': y_test,
        'feature_names': feature_names,
        'encoder': target_encoder,
        'scaler': scaler
    }

In [18]:
def prepare_data_with_onehot_encoder(df, test_size=0.2, random_state=42):

    numerical_features = ['bill_length_mm', 'bill_depth_mm',
                         'flipper_length_mm', 'body_mass_g']
    categorical_features = ['island']

    y = df['target'].values

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numerical_features),
            ('cat', OneHotEncoder(drop='first', sparse_output=False), categorical_features)
        ])

    X = df[numerical_features + categorical_features]

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state, stratify=y)

    X_train_processed = preprocessor.fit_transform(X_train)
    X_test_processed = preprocessor.transform(X_test)

    feature_names = (numerical_features + list(preprocessor.named_transformers_['cat'].get_feature_names_out(categorical_features)))

    print(f"\nИсходные категории island: {df['island'].unique()}")
    print(f"Итоговые признаки: {feature_names}")
    print(f"Размер X_train: {X_train_processed.shape}")
    print(f"Размер X_test: {X_test_processed.shape}")
    print(f"   One-Hot создал {len(feature_names) - len(numerical_features)} дополнительных признаков")

    return {
        'X_train': X_train_processed,
        'X_test': X_test_processed,
        'y_train': y_train,
        'y_test': y_test,
        'feature_names': feature_names,
        'preprocessor': preprocessor
    }

In [19]:
data_target = prepare_data_with_target_encoder(df)

Глобальное среднее целевой переменной: 0.4489
  Категория 'Torgersen':
    Количество: 51
    Среднее по категории: 0.0000
    Сглаженное среднее: 0.0401
  Категория 'Biscoe':
    Количество: 167
    Среднее по категории: 0.7365
    Сглаженное среднее: 0.7282
  Категория 'Dream':
    Количество: 56
    Среднее по категории: 0.0000
    Сглаженное среднее: 0.0368

Итоговые признаки: ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'island_target_encoded']
X_train: (219, 5)
X_test: (55, 5)


In [22]:
data_onehot = prepare_data_with_onehot_encoder(df)


Исходные категории island: ['Torgersen' 'Biscoe' 'Dream']
Итоговые признаки: ['bill_length_mm', 'bill_depth_mm', 'flipper_length_mm', 'body_mass_g', 'island_Dream', 'island_Torgersen']
Размер X_train: (219, 6)
Размер X_test: (55, 6)
   One-Hot создал 2 дополнительных признаков


In [24]:
def test_model(X_train, y_train, X_test, y_test, model_params, model_name):

    print(f"\n{model_name}")
    print("-" * 50)

    model = MyBinaryLogisticRegression(**model_params)
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)

    f1 = f1_score(y_test, y_pred)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"F1-Score: {f1:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Intercept: {model.intercept_:.4f}")

    print("Коэффициенты:")
    if 'feature_names' in globals():
        feature_names = data_target['feature_names'] if 'target' in model_name.lower() else data_onehot['feature_names']
        for name, coef in zip(feature_names, model.coef_):
            print(f"  {name}: {coef:.4f}")
    else:
        for i, coef in enumerate(model.coef_):
            print(f"  feature_{i}: {coef:.4f}")

    return {
        'model_name': model_name,
        'model': model,
        'f1_score': f1,
        'accuracy': accuracy,
        'params': model_params
    }


configurations = [
    # Градиентный спуск
    {'lr': 0.1, 'n_iter': 500, 'optimizer': 'gd', 'penalty': None,
     'lambda1': 0, 'lambda2': 0, 'batch_size': 32, 'random_state': 42,
     'name': 'GD без регуляризации'},

    {'lr': 0.1, 'n_iter': 500, 'optimizer': 'gd', 'penalty': 'l1',
     'lambda1': 0.1, 'lambda2': 0, 'batch_size': 32, 'random_state': 42,
     'name': 'GD с L1'},

    {'lr': 0.1, 'n_iter': 500, 'optimizer': 'gd', 'penalty': 'l2',
     'lambda1': 0, 'lambda2': 0.1, 'batch_size': 32, 'random_state': 42,
     'name': 'GD с L2'},

    {'lr': 0.1, 'n_iter': 500, 'optimizer': 'gd', 'penalty': 'l1l2',
     'lambda1': 0.05, 'lambda2': 0.05, 'batch_size': 32, 'random_state': 42,
     'name': 'GD с L1L2'},

    # Стохастический градиентный спуск
    {'lr': 0.05, 'n_iter': 50, 'optimizer': 'sgd', 'penalty': None,
     'lambda1': 0, 'lambda2': 0, 'batch_size': 32, 'random_state': 42,
     'name': 'SGD без регуляризации'},

    {'lr': 0.05, 'n_iter': 50, 'optimizer': 'sgd', 'penalty': 'l1',
     'lambda1': 0.1, 'lambda2': 0, 'batch_size': 32, 'random_state': 42,
     'name': 'SGD с L1'},

    {'lr': 0.05, 'n_iter': 50, 'optimizer': 'sgd', 'penalty': 'l2',
     'lambda1': 0, 'lambda2': 0.1, 'batch_size': 32, 'random_state': 42,
     'name': 'SGD с L2'},

    {'lr': 0.05, 'n_iter': 50, 'optimizer': 'sgd', 'penalty': 'l1l2',
     'lambda1': 0.05, 'lambda2': 0.05, 'batch_size': 32, 'random_state': 42,
     'name': 'SGD с L1L2'},

    # Метод Ньютона
    {'lr': 0.1, 'n_iter': 10, 'optimizer': 'newton', 'penalty': None,
     'lambda1': 0, 'lambda2': 0, 'batch_size': 32, 'random_state': 42,
     'name': 'Newton без регуляризации'},

    {'lr': 0.1, 'n_iter': 10, 'optimizer': 'newton', 'penalty': 'l2',
     'lambda1': 0, 'lambda2': 0.1, 'batch_size': 32, 'random_state': 42,
     'name': 'Newton с L2'},

    # Newton с L1 не рекомендуется из-за недифференцируемости
    {'lr': 0.1, 'n_iter': 10, 'optimizer': 'newton', 'penalty': 'l1l2',
     'lambda1': 0.05, 'lambda2': 0.05, 'batch_size': 32, 'random_state': 42,
     'name': 'Newton с L1L2'},
]

Все для Target Encoding

In [26]:
results_target = []
for config in configurations:
    if config['penalty'] == 'l1' and config['optimizer'] == 'newton':
        continue

    result = test_model(
        data_target['X_train'], data_target['y_train'],
        data_target['X_test'], data_target['y_test'],
        {k: v for k, v in config.items() if k != 'name'},
        f"Target Encoding - {config['name']}"
    )
    results_target.append(result)


Target Encoding - GD без регуляризации
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.5405
Коэффициенты:
  feature_0: 1.1825
  feature_1: -1.7207
  feature_2: 1.4568
  feature_3: 1.1534
  feature_4: 0.8514

Target Encoding - GD с L1
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.3520
Коэффициенты:
  feature_0: 0.2690
  feature_1: -0.8752
  feature_2: 0.8514
  feature_3: 0.2853
  feature_4: 0.0161

Target Encoding - GD с L2
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.3298
Коэффициенты:
  feature_0: 0.4429
  feature_1: -0.5736
  feature_2: 0.5281
  feature_3: 0.4408
  feature_4: 0.3787

Target Encoding - GD с L1L2
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.3243
Коэффициенты:
  feature_0: 0.4157
  feature_1: -0.6249
  feature_2: 0.5631
  feature_3: 0.4134
  feature_4: 0.3101



Все для one-hot

In [27]:
results_onehot = []
for config in configurations:
    if config['penalty'] == 'l1' and config['optimizer'] == 'newton':
        continue

    result = test_model(
        data_onehot['X_train'], data_onehot['y_train'],
        data_onehot['X_test'], data_onehot['y_test'],
        {k: v for k, v in config.items() if k != 'name'},
        f"One-Hot Encoding - {config['name']}"
    )
    results_onehot.append(result)


One-Hot Encoding - GD без регуляризации
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.3774
Коэффициенты:
  feature_0: 1.2510
  feature_1: -1.8645
  feature_2: 1.5271
  feature_3: 1.2339
  feature_4: -0.4037
  feature_5: -0.4645

One-Hot Encoding - GD с L1
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.3524
Коэффициенты:
  feature_0: 0.2724
  feature_1: -0.8880
  feature_2: 0.8409
  feature_3: 0.2895
  feature_4: -0.0002
  feature_5: -0.0021

One-Hot Encoding - GD с L2
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.2474
Коэффициенты:
  feature_0: 0.4774
  feature_1: -0.6202
  feature_2: 0.5617
  feature_3: 0.4738
  feature_4: -0.1293
  feature_5: -0.1285

One-Hot Encoding - GD с L1L2
--------------------------------------------------
F1-Score: 1.0000
Accuracy: 1.0000
Intercept: -0.2182
Коэффициенты:
  feature_0: 0.4715
  feature_1:

In [32]:
comparison_data = []

for result in results_target:
    comparison_data.append({
        'Метод кодирования': 'Target',
        'Оптимизатор': result['params']['optimizer'].upper(),
        'Регуляризация': result['params']['penalty'] if result['params']['penalty'] else 'нет',
        'F1-Score': result['f1_score'],
        'Accuracy': result['accuracy'],
        'Коэффициент L1': result['params']['lambda1'],
        'Коэффициент L2': result['params']['lambda2']
    })

for result in results_onehot:
    comparison_data.append({
        'Метод кодирования': 'One-Hot',
        'Оптимизатор': result['params']['optimizer'].upper(),
        'Регуляризация': result['params']['penalty'] if result['params']['penalty'] else 'нет',
        'F1-Score': result['f1_score'],
        'Accuracy': result['accuracy'],
        'Коэффициент L1': result['params']['lambda1'],
        'Коэффициент L2': result['params']['lambda2']
    })

comparison_df = pd.DataFrame(comparison_data)

print(comparison_df.sort_values('F1-Score', ascending=False).to_string(index=False))


Метод кодирования Оптимизатор Регуляризация  F1-Score  Accuracy  Коэффициент L1  Коэффициент L2
           Target          GD           нет       1.0       1.0            0.00            0.00
           Target          GD            l1       1.0       1.0            0.10            0.00
           Target          GD            l2       1.0       1.0            0.00            0.10
           Target          GD          l1l2       1.0       1.0            0.05            0.05
           Target         SGD           нет       1.0       1.0            0.00            0.00
           Target         SGD            l1       1.0       1.0            0.10            0.00
           Target         SGD            l2       1.0       1.0            0.00            0.10
           Target         SGD          l1l2       1.0       1.0            0.05            0.05
           Target      NEWTON           нет       1.0       1.0            0.00            0.00
           Target      NEWTON           

Лучшая модель с target encoding

In [30]:
max(results_target, key=lambda x: x['f1_score'])

{'model_name': 'Target Encoding - GD без регуляризации',
 'model': <__main__.MyBinaryLogisticRegression at 0x782519138320>,
 'f1_score': 1.0,
 'accuracy': 1.0,
 'params': {'lr': 0.1,
  'n_iter': 500,
  'optimizer': 'gd',
  'penalty': None,
  'lambda1': 0,
  'lambda2': 0,
  'batch_size': 32,
  'random_state': 42}}

Лучшая модель с one-hot encoding

In [33]:
max(results_onehot, key=lambda x: x['f1_score'])

{'model_name': 'One-Hot Encoding - GD без регуляризации',
 'model': <__main__.MyBinaryLogisticRegression at 0x7825191a7b90>,
 'f1_score': 1.0,
 'accuracy': 1.0,
 'params': {'lr': 0.1,
  'n_iter': 500,
  'optimizer': 'gd',
  'penalty': None,
  'lambda1': 0,
  'lambda2': 0,
  'batch_size': 32,
  'random_state': 42}}

# Теоретическая часть

Пусть данные имеют вид
$$
(x_i, y_i), \quad y_i \in \{1, \ldots,M\}, \quad i \in \{1, \ldots, N\},
$$
причем первая координата набора признаков каждого объекта равна $1$.
Используя `softmax`-подход, дискриминативная модель имеет следующий вид
$$
\mathbb P(C_k|x) = \frac{\exp(\omega_k^Tx)}{\sum_i \exp(\omega_i^Tx)}.
$$
Для написания правдоподобия удобно провести `one-hot` кодирование меток класса, сопоставив каждому объекту $x_i$ вектор $\widehat y_i = (y_{11}, \ldots, y_{1M})$ длины $M$, состоящий из нулей и ровно одной единицы ($y_{iy_i} = 1$), отвечающей соответствующему классу. В этом случае правдоподобие имеет вид
$$
\mathbb P(D|\omega) = \prod_{i = 1}^{N}\prod_{j = 1}^M \mathbb P(C_j|x_i)^{y_{ij}}.
$$
Ваша задача: вывести функцию потерь, градиент и гессиан для многоклассовой логистической регрессии. Реализовать матрично. На синтетическом примере продемонстрировать работу алгоритма, построить гиперплоскости, объяснить классификацию